#Google Drive Mount and Access to Datathon Directory


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/Datathon/datasets

/content/drive/MyDrive/Datathon/datasets


#Library Calls

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
from tabulate import tabulate
from IPython.display import display
from google.colab import files
import re
import csv
import openpyxl

#Data Extraction

##Labor Force Data

###Transformation Functions

In [50]:
def get_gender(group):
    if isinstance(group, str):  # Check if the value is a string
        group_lower = group.lower()
        if 'women' in group_lower:
            return 'Women'
        elif 'men' in group_lower:
            return 'Men'
    return 'Total'  # Return Total if the value is not a string (like NaN)

def get_race(group):
    if isinstance(group, str):
        group_lower = group.lower()
        if 'white' in group_lower:
            return 'White'
        elif 'black' in group_lower:
            return 'Black'
        elif 'asian' in group_lower:
            return 'Asian'
        elif 'hispanic' in group_lower:
            return 'Hispanic'
        elif 'mixed' in group_lower:
            return 'Mixed'
    return 'Total'


def get_age(group):
    if isinstance(group, str):
        if '16 to 19' in group:
            return '16-19'
        elif '20 to 24' in group:
            return '20-24'
        elif '25 to 34' in group:
            return '25-34'
        elif '35 to 44' in group:
            return '35-44'
        elif '45 to 54' in group:
            return '45-54'
        elif '55 to 64' in group:
            return '55-64'
        elif '65 years and over' in group:
            return '65+'
        elif '16 to 24' in group:
            return '16-24'
        elif '25 to 54' in group:
            return '25-54'
        elif '55' in group:
            return '55+'
    return 'Total'

def clean_excel_to_table(file_path):
    """
    Removes footnotes from an Excel file.

    Parameters:
    - file_path (str): Path to the Excel file.

    Returns:
    - str: Path to the cleaned Excel file.
    """

    # Load the Excel workbook
    wb = openpyxl.load_workbook(file_path)

    # Loop through all the sheets in the workbook
    for sheet_name in wb.sheetnames:
        sheet = wb[sheet_name]

        # List to store modified merged cell ranges
        new_merged_cells = []

                # Adjust all merged cell ranges
        for merged_cell in list(sheet.merged_cells):
            # Convert the column indices to column letters
            min_col_letter = openpyxl.utils.get_column_letter(merged_cell.min_col)
            max_col_letter = openpyxl.utils.get_column_letter(merged_cell.max_col)

            # Adjust the merged cell range by reducing the row indices by 3
            # Store the original value
            original_value = sheet.cell(row=merged_cell.min_row, column=merged_cell.min_col).value
            new_min_row = merged_cell.min_row - 3
            new_max_row = merged_cell.max_row - 3

            if new_min_row >= 1:
                new_range = f"{min_col_letter}{new_min_row}:{max_col_letter}{new_max_row}"
                new_merged_cells.append((new_range, original_value))

            # Unmerge the original merged cell range
            sheet.unmerge_cells(str(merged_cell))

        # Delete the first three rows after adjusting all the merged cell ranges
        sheet.delete_rows(1, 3)

        # Delete columns L, M, and N (error range)
        sheet.delete_cols(12, 3)  # Delete 3 columns starting from column L (12)

        # After adjusting all the merged cell ranges, merge the cells with the updated values
        for new_range, original_value in new_merged_cells:
            start_cell = new_range.split(":")[0]
            sheet[start_cell].value = original_value
            sheet.merge_cells(new_range)

        # Iterate through the rows to find the starting point of footnotes
        for row in sheet.iter_rows():
            # If the cell contains the "SOURCE" keyword
            if row[0].value and "SOURCE:" in str(row[0].value):
                # Delete rows from the current to the end of the sheet
                for delete_row in range(row[0].row, sheet.max_row + 1):
                    sheet.delete_rows(row[0].row)
                break

    # Define the name for the cleaned file
    cleaned_file_path = 'cleaned_' + file_path.split("/")[-1]
    wb.save(cleaned_file_path)

    return cleaned_file_path

#DataFrame Creation
def read_and_combine_headers(excel_path):
    # Read the Excel file with 3 levels of headers
    df = pd.read_excel(excel_path, header=[0, 1, 2])

    # Join the multi-level headers into a single header
    df.columns = [' '.join(col).strip() for col in df.columns.values]

    return df

# Function to clean the header names
def clean_header(header):
    return tuple('' if 'Unnamed' in level else level for level in header)

def excel_to_concatenated_dataframe(file_path):
    """
    Reads through the sheets of an Excel file (excluding the first sheet)
    and returns a concatenated pandas DataFrame with combined headers.

    Parameters:
    - file_path: Path to the Excel file

    Returns:
    - Concatenated pandas DataFrame of all sheets except the first one, concatenated vertically
    """
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # List to store DataFrames for each sheet
    dfs = []

    # Iterate through the sheets in the Excel file, skipping the first sheet
    for sheet_name in xls.sheet_names:
        # Read the sheet into a DataFrame with multi-level headers
        df = pd.read_excel(xls, sheet_name, header=[0, 1, 2])

        # Apply the function clean header to each column
        df.columns = df.columns.to_series().apply(clean_header)

        # Combine the multi-level headers into a single level
        df.columns = [' '.join(col).strip() for col in df.columns.values]

        # Add the 'year' column with the sheet name as the value (converted to numeric)
        try:
            year_value = int(sheet_name)
        except ValueError:
            year_value = sheet_name

        df['year'] = year_value

        dfs.append(df)

    #Vertically concatenate the DataFrames
    concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)

    return concatenated_df



###Transformation Execution

In [41]:
# clean sheets:
cleaned_path = clean_excel_to_table('Unemployment by State and Race.xlsx')
print(f"Cleaned file saved at: {cleaned_path}")

Cleaned file saved at: cleaned_Unemployment by State and Race.xlsx


In [51]:
#concatenate dataframes
df = excel_to_concatenated_dataframe('cleaned_Unemployment by State and Race.xlsx')
display(df.head())

,State FIPS Code,Group Code,State,Group,Civilian non-institutional population,Civilian labor force Number,Civilian labor force Percent of population,Civilian labor force Employment Number,Civilian labor force Employment Percent of population,Civilian labor force Unemployment Number,Civilian labor force Unemployment Rate,Civilian labor force Unemployment Error range,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,of rate 1,2000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
2,1.0,1.0,Alabama,Total,3401.0,2154.0,63.3,2055.0,60.4,99,4.6,NaN,2000
3,1.0,2.0,Alabama,Men,1589.0,1124.0,70.7,1074.0,67.6,50,4.5,NaN,2000
4,1.0,3.0,Alabama,Women,1812.0,1030.0,56.9,981.0,54.1,49,4.8,NaN,2000


In [52]:
# Apply functions to extract Gender, Race, and Age
df['Gender'] = df['Group'].apply(get_gender)
df['Race'] = df['Group'].apply(get_race)
df['Age'] = df['Group'].apply(get_age)

# Drop the original 'Group' column
df = df.drop('Group', axis=1)

# Drop columns related to "Error Range"
df = df.drop(columns=[col for col in df.columns if "Error" in col])
df = df.drop(columns=['State FIPS Code', 'Group Code'])

# Rename variable headers
rename_dict = {
    'Civilian non-institutional population': 'civ-non-instit-pop',
    'Civilian labor force Number': 'civ-lab-force-num',
    'Civilian labor force Employment Number': 'civ-lab-force-emp-num',
    'Civilian labor force Employment Percentage of population': 'civ-lab-force-emp-perc-pop',
    'Civilian labor force Unemployment Number': 'civ-lab-force-unemp-num',
    'Civilian labor force Unemployment Rate': 'civ-lab-force-unemp-rate',
}
df.rename(columns=rename_dict, inplace=True)

In [53]:
#drop NaN rows
df = df.dropna(subset=['State'])

display(df.head())

,State,civ-non-instit-pop,civ-lab-force-num,Civilian labor force Percent of population,civ-lab-force-emp-num,Civilian labor force Employment Percent of population,civ-lab-force-unemp-num,civ-lab-force-unemp-rate,year,Gender,Race,Age
2,Alabama,3401.0,2154.0,63.3,2055.0,60.4,99,4.6,2000,Total,Total,Total
3,Alabama,1589.0,1124.0,70.7,1074.0,67.6,50,4.5,2000,Men,Total,Total
4,Alabama,1812.0,1030.0,56.9,981.0,54.1,49,4.8,2000,Women,Total,Total
5,Alabama,2482.0,1585.0,63.8,1532.0,61.7,53,3.3,2000,Total,White,Total
6,Alabama,1179.0,859.0,72.8,834.0,70.7,25,2.9,2000,Men,White,Total


In [54]:
#Downloading the labor force data
df.to_csv('data-labor-force.csv', index=False)

##R%D Obligation Data

###Transformation Function

In [ ]:
def process_excel_sheets(file_path):
    """
    Reads sheets from an Excel file, melts them based on specified structure, and then concatenates them horizontally.

    Parameters:
    - file_path (str): Path to the Excel file.

    Returns:
    - DataFrame: Processed DataFrame.
    """

    # Load all sheets from the Excel file into a dictionary of DataFrames
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    processed_dfs = []

    # Process each sheet individually
    for sheet_name, df in all_sheets.items():
        # Melt the DataFrame
        df_melted = df.melt(id_vars=["State"],
                            var_name="year",
                            value_name=sheet_name)
        processed_dfs.append(df_melted)

    # Start with the first processed DataFrame
    final_df = processed_dfs[0]

    # Merge the rest of the processed DataFrames with the initial one based on 'State' and 'year'
    for df in processed_dfs[1:]:
        final_df = final_df.merge(df, on=['State', 'year'], how='outer')

    return final_df

###Transformation Execution

In [ ]:
#usage
file_path = 'se-rd-fund.xlsx'
result_df = process_excel_sheets(file_path)
display(result_df.head())

,State,year,fed-rd-obligations-dollar-thous,individuals-se-occupations,fed-rd-obligations-by-indiv-se-
0,United States,2003,101366433.0,4961540.0,20430.437525
1,Alabama,2003,3211827.0,56380.0,56967.488471
2,Alaska,2003,399401.0,10600.0,37679.339623
3,Arizona,2003,2385404.0,92120.0,25894.528875
4,Arkansas,2003,145361.0,21340.0,6811.668229


In [ ]:
df = result_df

#Downloading the data
df.to_csv('data-se-rd-fund.csv', index=False)

##Patents Data

###Data Transformation

In [ ]:
#ussing the excel processing function
file_path = 'se-patents.xlsx'
result_df = process_excel_sheets(file_path)
display(result_df.head())



,State,year,patents-awarded,individuals-in-se-occupations,patents-pe-1000-individuals-s&e
0,United States,2003,87604,4961540.0,17.656615
1,Alabama,2003,407,56380.0,7.218872
2,Alaska,2003,38,10600.0,3.584906
3,Arizona,2003,1563,92120.0,16.967000
4,Arkansas,2003,158,21340.0,7.403936


In [ ]:
df = result_df

#Downloading the data
df.to_csv('data-se-patents.csv', index=False)

##Foreign Born Workers in S%E Data

###Data Transformation

In [ ]:
#Using function defined in fund related data transformation (se obligations)
file_path = 'se-foreign-born-workers.xlsx'
result_df = process_excel_sheets(file_path)
display(result_df.head())

,State,year,foreign-born-workers-se-occ,all-workers-in-se-occupations,perc-foreign-born-se-workers
0,United States,2005,1079655,5319585,20.295850
1,Alabama,2005,4655,61985,7.509881
2,Alaska,2005,1110,12375,8.969697
3,Arizona,2005,15565,101355,15.356914
4,Arkansas,2005,2160,24710,8.741400


In [ ]:
df = result_df

#Downloading he data
df.to_csv('data-se-foreign-born-workers.csv', index=False)

##Federal R&D as Percentage of GDP

In [ ]:
#Using function defined in fund related data transformation (se obligations and patents)
file_path = 'rd-performance-to-state-gdp.xlsx'
result_df = process_excel_sheets(file_path)
display(result_df.head())

,State,year,rd-performed-dollar-millions,state-gdop-dollar-millions,rd-perform-perc-of-gdp
0,United States,1991,160876.0,5875406,2.738126
1,Alabama,1991,1511.0,76043,1.987034
2,Alaska,1991,146.0,22283,0.655208
3,Arizona,1991,1399.0,73358,1.907086
4,Arkansas,1991,198.0,41572,0.476282


In [ ]:
df = result_df

#Downloading he data
df.to_csv('data-rd-performance-to-state-gdp.csv', index=False)

##Graduation Rate

###Transformation Execution

In [4]:
#importing the data
df = pd.read_excel('completion-by-state-and-ethnicity.xlsx')
display(df.head())

,Cohort Year,Sector,Race/Ethnicity,State,Total Completion (%),Completion at Same Institution (%),Completion at Different 4YR (%),Completion at Different 2YR (%),"Still Enrolled (Anywhere, %)","No Longer Enrolled (Anywhere, %)"
0,2015.0,Public Four-Year,Asian,National,80.9506,71.147259,7.662499,2.140841,8.320531,10.728869
1,NaN,NaN,NaN,Alabama,71.823785,62.816768,8.776073,0.230944,6.484284,21.69193
2,NaN,NaN,NaN,Alaska,††,††,††,††,††,††
3,NaN,NaN,NaN,Arizona,82.905083,75.455605,4.597419,2.852059,8.230957,8.86396
4,NaN,NaN,NaN,Arkansas,76.557075,68.389931,7.227484,0.939661,6.302877,17.140048


In [5]:
# Filling the NaN values
df['Cohort Year'] = df['Cohort Year'].fillna(method='ffill')
df['Sector'] = df['Sector'].fillna(method='ffill')
df['Race/Ethnicity'] = df['Race/Ethnicity'].fillna(method='ffill')

display(df)

,Cohort Year,Sector,Race/Ethnicity,State,Total Completion (%),Completion at Same Institution (%),Completion at Different 4YR (%),Completion at Different 2YR (%),"Still Enrolled (Anywhere, %)","No Longer Enrolled (Anywhere, %)"
0,2015.0,Public Four-Year,Asian,National,80.9506,71.147259,7.662499,2.140841,8.320531,10.728869
1,2015.0,Public Four-Year,Asian,Alabama,71.823785,62.816768,8.776073,0.230944,6.484284,21.69193
2,2015.0,Public Four-Year,Asian,Alaska,††,††,††,††,††,††
3,2015.0,Public Four-Year,Asian,Arizona,82.905083,75.455605,4.597419,2.852059,8.230957,8.86396
4,2015.0,Public Four-Year,Asian,Arkansas,76.557075,68.389931,7.227484,0.939661,6.302877,17.140048
...,...,...,...,...,...,...,...,...,...,...
1843,2012.0,Public Four-Year,White,Texas,72.53,56.2,12.45,3.88,10.39,17.09
1844,2012.0,Public Four-Year,White,Vermont,84.82,74.02,9.54,1.26,5.44,9.75
1845,2012.0,Public Four-Year,White,Virginia,87.01,78.2,6.59,2.22,4.85,8.13
1846,2012.0,Public Four-Year,White,Washington,68.52,57.11,8.52,2.89,9.19,22.29


In [6]:
# Remove rows with NaN values
df_cleaned = df.dropna()

# Remove rows containing '††' or '*'
df_cleaned = df_cleaned[~df_cleaned.isin(['††', '*']).any(axis=1)]

display(df_cleaned)

df = df_cleaned

,Cohort Year,Sector,Race/Ethnicity,State,Total Completion (%),Completion at Same Institution (%),Completion at Different 4YR (%),Completion at Different 2YR (%),"Still Enrolled (Anywhere, %)","No Longer Enrolled (Anywhere, %)"
0,2015.0,Public Four-Year,Asian,National,80.9506,71.147259,7.662499,2.140841,8.320531,10.728869
1,2015.0,Public Four-Year,Asian,Alabama,71.823785,62.816768,8.776073,0.230944,6.484284,21.69193
3,2015.0,Public Four-Year,Asian,Arizona,82.905083,75.455605,4.597419,2.852059,8.230957,8.86396
4,2015.0,Public Four-Year,Asian,Arkansas,76.557075,68.389931,7.227484,0.939661,6.302877,17.140048
5,2015.0,Public Four-Year,Asian,California,85.456071,78.979202,4.30074,2.176128,7.818731,6.725198
...,...,...,...,...,...,...,...,...,...,...
1843,2012.0,Public Four-Year,White,Texas,72.53,56.2,12.45,3.88,10.39,17.09
1844,2012.0,Public Four-Year,White,Vermont,84.82,74.02,9.54,1.26,5.44,9.75
1845,2012.0,Public Four-Year,White,Virginia,87.01,78.2,6.59,2.22,4.85,8.13
1846,2012.0,Public Four-Year,White,Washington,68.52,57.11,8.52,2.89,9.19,22.29


In [9]:
#Replacing Values
df['State'] = df['State'].replace('National', 'United States')
df['Race'] = df['Race'].replace('Latinx', 'Hispanic')

#Rename variables
df.rename(columns={'Cohort Year': 'year'}, inplace=True)
df.rename(columns={'Race/Ethnicity': 'Race'}, inplace=True)


display(df)


,year,Sector,Race,State,Total Completion (%),Completion at Same Institution (%),Completion at Different 4YR (%),Completion at Different 2YR (%),"Still Enrolled (Anywhere, %)","No Longer Enrolled (Anywhere, %)"
0,2015.0,Public Four-Year,Asian,United States,80.9506,71.147259,7.662499,2.140841,8.320531,10.728869
1,2015.0,Public Four-Year,Asian,Alabama,71.823785,62.816768,8.776073,0.230944,6.484284,21.69193
3,2015.0,Public Four-Year,Asian,Arizona,82.905083,75.455605,4.597419,2.852059,8.230957,8.86396
4,2015.0,Public Four-Year,Asian,Arkansas,76.557075,68.389931,7.227484,0.939661,6.302877,17.140048
5,2015.0,Public Four-Year,Asian,California,85.456071,78.979202,4.30074,2.176128,7.818731,6.725198
...,...,...,...,...,...,...,...,...,...,...
1843,2012.0,Public Four-Year,White,Texas,72.53,56.2,12.45,3.88,10.39,17.09
1844,2012.0,Public Four-Year,White,Vermont,84.82,74.02,9.54,1.26,5.44,9.75
1845,2012.0,Public Four-Year,White,Virginia,87.01,78.2,6.59,2.22,4.85,8.13
1846,2012.0,Public Four-Year,White,Washington,68.52,57.11,8.52,2.89,9.19,22.29


In [10]:
# Write to a CSV file
df.to_csv('data-completion-rates.csv', index=False)

#Merging Compatible Data

##Federal Fund Data

In [ ]:
def merge_csv_files_on_state_year(file_paths):
    """
    Reads CSV files from given file paths and merges them based on columns 'state' and 'year'.

    Parameters:
    - file_paths (list): List of paths to the CSV files.

    Returns:
    - DataFrame: Merged DataFrame.
    """

    # Initialize the first DataFrame from the list
    master_df = pd.read_csv(file_paths[0])

    # Iterate through the remaining file paths and merge with the master DataFrame
    for file_path in file_paths[1:]:
        df = pd.read_csv(file_path)
        master_df = pd.merge(master_df, df, on=['State', 'year'], how='outer')

    return master_df


# Execution:
file_paths = [
    "data-se-rd-fund.csv",
    "data-se-patents.csv",
    "data-se-foreign-born-workers.csv",
    "data-rd-performance-to-state-gdp.csv"
]
result_df = merge_csv_files_on_state_year(file_paths)
display(result_df.head())

,State,year,fed-rd-obligations-dollar-thous,individuals-se-occupations,fed-rd-obligations-by-indiv-se-,patents-awarded,individuals-in-se-occupations,patents-pe-1000-individuals-s&e,foreign-born-workers-se-occ,all-workers-in-se-occupations,perc-foreign-born-se-workers,rd-performed-dollar-millions,state-gdop-dollar-millions,rd-perform-perc-of-gdp
0,United States,2003,101366433.0,4961540.0,20430.437525,87604.0,4961540.0,17.656615,NaN,NaN,NaN,291365.0,11532329.0,2.526506
1,Alabama,2003,3211827.0,56380.0,56967.488471,407.0,56380.0,7.218872,NaN,NaN,NaN,2543.0,134153.0,1.895597
2,Alaska,2003,399401.0,10600.0,37679.339623,38.0,10600.0,3.584906,NaN,NaN,NaN,321.0,32038.0,1.001935
3,Arizona,2003,2385404.0,92120.0,25894.528875,1563.0,92120.0,16.967000,NaN,NaN,NaN,3578.0,193635.0,1.847806
4,Arkansas,2003,145361.0,21340.0,6811.668229,158.0,21340.0,7.403936,NaN,NaN,NaN,509.0,78695.0,0.646801


In [ ]:
df = result_df

#Downloading the data in drive
df.to_csv('Master-federal-funds.csv', index=False)

##Labor Force + Completion Rate data

In [15]:
def merge_csv_files_on_state_year_race(file_paths):
    """
    Reads CSV files from given file paths and merges them based on columns 'state' and 'year'.

    Parameters:
    - file_paths (list): List of paths to the CSV files.

    Returns:
    - DataFrame: Merged DataFrame.
    """

    # Initialize the first DataFrame from the list and filter it
    master_df = pd.read_csv(file_paths[0])
    master_df = master_df[(master_df['Gender'] == 'Total') & (master_df['Age'] == 'Total')]

    # Iterate through the remaining file paths and merge with the filtered master DataFrame
    for file_path in file_paths[1:]:
        df = pd.read_csv(file_path)
        master_df = pd.merge(master_df, df, on=['State', 'year', 'Race'], how='outer')

    return master_df


# Execution:
file_paths = [
    "data-labor-force.csv",
    "data-completion-rates.csv"
]
result_df = merge_csv_files_on_state_year_race(file_paths)
display(result_df.head())

,State,civ-non-instit-pop,civ-lab-force-num,Civilian labor force Percent of population,civ-lab-force-emp-num,Civilian labor force Employment Percent of population,civ-lab-force-unemp-num,civ-lab-force-unemp-rate,year,Gender,Race,Age,Sector,Total Completion (%),Completion at Same Institution (%),Completion at Different 4YR (%),Completion at Different 2YR (%),"Still Enrolled (Anywhere, %)","No Longer Enrolled (Anywhere, %)"
0,Alabama,3401.0,2154.0,63.3,2055.0,60.4,99,4.6,2000.0,Total,Total,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,2482.0,1585.0,63.8,1532.0,61.7,53,3.3,2000.0,Total,White,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,890.0,551.0,61.8,505.0,56.7,46,8.3,2000.0,Total,Black,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alaska,438.0,322.0,73.5,301.0,68.6,21,6.6,2000.0,Total,Total,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alaska,342.0,257.0,74.9,243.0,70.9,14,5.4,2000.0,Total,White,Total,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df = result_df

#Downloading the data in drive
df.to_csv('Master-labor-and-graduation.csv', index=False)

##All Sets

In [6]:
def merge_csv_files_final(file_paths):
    """
    Reads CSV files from given file paths and merges them based on columns 'state' and 'year'.

    Parameters:
    - file_paths (list): List of paths to the CSV files.

    Returns:
    - DataFrame: Merged DataFrame.
    """

    # Initialize the first DataFrame from the list and filter it
    master_df = pd.read_csv(file_paths[0])

    # Iterate through the remaining file paths and merge with the filtered master DataFrame
    for file_path in file_paths[1:]:
        df = pd.read_csv(file_path)
        master_df = pd.merge(master_df, df, on=['State', 'year', 'Race', 'Gender', 'Age'], how='outer')

    return master_df


df = pd.read_csv('Master-federal-funds.csv')
df['Age'] = 'Total'
df['Gender'] = 'Total'
df['Race'] = 'Total'
#Downloading the data in drive
df.to_csv('Master-federal-funds-total.csv', index=False)

df = pd.read_csv('data-completion-rates.csv')
df['Age'] = 'Total'
df['Gender'] = 'Total'
#Downloading the data in drive
df.to_csv('data-completion-rates-total.csv', index=False)



# Execution:
file_paths = [
    "data-labor-force.csv",
    "data-completion-rates-total.csv",
    'Master-federal-funds-total.csv'
]
result_df = merge_csv_files_final(file_paths)
display(result_df.head())

,State,civ-non-instit-pop,civ-lab-force-num,Civilian labor force Percent of population,civ-lab-force-emp-num,Civilian labor force Employment Percent of population,civ-lab-force-unemp-num,civ-lab-force-unemp-rate,year,Gender,...,fed-rd-obligations-by-indiv-se-,patents-awarded,individuals-in-se-occupations,patents-pe-1000-individuals-s&e,foreign-born-workers-se-occ,all-workers-in-se-occupations,perc-foreign-born-se-workers,rd-performed-dollar-millions,state-gdop-dollar-millions,rd-perform-perc-of-gdp
0,Alabama,3401.0,2154.0,63.3,2055.0,60.4,99,4.6,2000.0,Total,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1730.0,119852.0,1.443447
1,Alabama,1589.0,1124.0,70.7,1074.0,67.6,50,4.5,2000.0,Men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alabama,1812.0,1030.0,56.9,981.0,54.1,49,4.8,2000.0,Women,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alabama,2482.0,1585.0,63.8,1532.0,61.7,53,3.3,2000.0,Total,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alabama,1179.0,859.0,72.8,834.0,70.7,25,2.9,2000.0,Men,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df = result_df

df = df.drop(columns=['individuals-in-se-occupations', 'individuals-se-occupations'])


#Downloading the data in drive
df.to_csv('Master-final.csv', index=False)